In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [29]:
custinfo = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_custinfo_full_hashed.csv')
cdtx = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_cdtx0001_full_hashed.csv')
dp = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_dp_full_hashed.csv')
remit = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_remit1_full_hashed.csv')
alert_pub = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_x_alert_date.csv')
alert_train = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/train_x_alert_date.csv')
alert =pd.concat([alert_pub, alert_train])
label = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/train_y_answer.csv')

In [30]:
custs = custinfo.cust_id.unique()

In [31]:
def process_date(x):
    dates = x[['date', 'tx_date', 'trans_date']].values
    u_dates = pd.unique(dates)
    u_dates = u_dates[~pd.isna(u_dates)]
    #print(u_dates)
    x['date'] = u_dates[0]
    return x

In [37]:
all_data = []
n_aks = 0
missed = []
for cust in tqdm(custs):
    dp_c = dp[dp.cust_id == cust]
    cdtx_c = cdtx[cdtx.cust_id == cust]
    remit_c = remit[remit.cust_id == cust]
    cust_data = pd.concat([cdtx_c, dp_c, remit_c], axis = 0)
    if len(cust_data) > 0:
        cust_data = cust_data.apply(lambda x: process_date(x), axis=1)
        cust_data.drop(columns = ['tx_date', 'trans_date'], inplace = True)
        #cust_data.fillna(-1, inplace = True)
        cust_data.sort_values(['date'], inplace = True)
        cust_data['alert_key'] = np.nan
        aks = custinfo[custinfo.cust_id == cust].alert_key # alert keys
        ads = alert[alert.alert_key.isin(aks)] # alert key-date mapping
        if len(ads) > 0:
            ads.sort_values(by=['date'], ascending=False, inplace=True)
            ads.drop_duplicates(inplace=True)
            n_aks += len(ads)
            for k, d in ads.values:
                idx = cust_data[cust_data.date<=d].index
                if len(idx) > 0:
                    cust_data.loc[idx, 'alert_key'] = k   
                else:
                    missed.append([cust, k, d])
        cust_data['alert_key'].fillna(method='bfill', inplace=True)
        cust_data.dropna(subset=['alert_key'], inplace=True)
        all_data.append(cust_data)

all_data = pd.concat(all_data)


  0%|          | 0/7708 [00:00<?, ?it/s]c:\Users\PoChen\anaconda3\envs\fintech\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
100%|██████████| 7708/7708 [41:51<00:00,  3.07it/s]   


In [41]:
custinfo

,alert_key,cust_id,risk_rank,occupation_code,total_asset,AGE
0,352249,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,1,19.0,1465816.0,7
1,352253,b212d14cb35676926682b2cf849e295d948888f556c07e...,1,2.0,98177.0,2
2,352254,e5b0002791c7852644a2730abeaa893cdf14a072ef7812...,1,19.0,2052922.0,7
3,352280,74214c478dc6519fbefe4bc31693865bdcd698ab974b64...,3,15.0,201906.0,5
4,352282,0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...,1,12.0,7450.0,5
...,...,...,...,...,...,...
25746,352123,b600c0720096c241f5aec16b096c6a353492eee8a5855a...,1,17.0,12207.0,2
25747,352124,06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a...,1,17.0,259985.0,4
25748,352125,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3
25749,352128,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,3,19.0,21647.0,4


In [39]:
n_aks

24157

In [40]:
len(missed)

342

In [42]:
missed[0]

['bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f2528254e87100b57ea',
 334129,
 330]

In [43]:
all_data[all_data.cust_id==missed[0][0]]

,cust_id,date,country,cur_type,amt,debit_credit,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,trans_no,trade_amount_usd,alert_key
332605,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,336.0,NaN,NaN,NaN,CR,14.0,1.0,4151.0,1.0,12.0,NaN,NaN,0.0,0.0,NaN,NaN,340321.0
332614,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,336.0,NaN,NaN,NaN,CR,14.0,1.0,5119.0,1.0,12.0,NaN,NaN,0.0,0.0,NaN,NaN,340321.0
332613,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,337.0,NaN,NaN,NaN,CR,1.0,3.0,278.0,1.0,16.0,NaN,NaN,0.0,0.0,NaN,NaN,340321.0
332615,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,347.0,NaN,NaN,NaN,DB,20.0,1.0,232578.0,1.0,13.0,21.0,110.0,0.0,1.0,NaN,NaN,353835.0
332616,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,347.0,NaN,NaN,NaN,DB,20.0,1.0,154.0,1.0,13.0,21.0,110.0,0.0,1.0,NaN,NaN,353835.0
332611,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,370.0,NaN,NaN,NaN,DB,10.0,2.0,136985.0,1.0,13.0,4.0,0.0,0.0,1.0,NaN,NaN,353835.0
332608,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,371.0,NaN,NaN,NaN,CR,7.0,2.0,801232.0,1.0,13.0,NaN,309.0,1.0,1.0,NaN,NaN,353835.0
332609,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,372.0,NaN,NaN,NaN,DB,14.0,2.0,1244851.0,1.0,13.0,3.0,310.0,1.0,1.0,NaN,NaN,353835.0
332612,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,372.0,NaN,NaN,NaN,DB,14.0,2.0,2037218.0,1.0,13.0,3.0,310.0,1.0,1.0,NaN,NaN,353835.0
332610,bc2d8f0134895f6deaee4eb189e06d8a81a5c25443559f...,376.0,NaN,NaN,NaN,DB,13.0,2.0,2057.0,1.0,13.0,4.0,0.0,0.0,1.0,NaN,NaN,355731.0


In [54]:
all_data

,cust_id,date,country,cur_type,amt,debit_credit,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,trans_no,trade_amount_usd,alert_key
278559,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,CR,13.0,2.0,6108.0,28.555,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
279501,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,DB,14.0,2.0,6575.0,28.535,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
278654,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,DB,15.0,2.0,18451.0,28.495,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
279533,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,DB,18.0,2.0,826714.0,1.000,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
279536,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,CR,19.0,2.0,53434.0,28.495,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091070,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,NaN,NaN,NaN,CR,14.0,2.0,51329.0,1.000,13.0,4.0,142.0,1.0,1.0,NaN,NaN,352111.0
544141,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,130.0,47.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352111.0
1091094,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,NaN,NaN,NaN,DB,1.0,2.0,719.0,1.000,16.0,NaN,NaN,0.0,0.0,NaN,NaN,352111.0
1091091,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,NaN,NaN,NaN,DB,12.0,2.0,2987.0,1.000,12.0,NaN,NaN,0.0,0.0,NaN,NaN,352111.0


In [44]:
all_data.alert_key.unique().shape

(21909,)

In [58]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
all_data.loc[:,'debit_credit'] = le.fit_transform(all_data.debit_credit)

In [61]:
all_data

,cust_id,date,country,cur_type,amt,debit_credit,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,trans_no,trade_amount_usd,alert_key
278559,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,0,13.0,2.0,6108.0,28.555,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
279501,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,1,14.0,2.0,6575.0,28.535,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
278654,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,1,15.0,2.0,18451.0,28.495,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
279533,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,1,18.0,2.0,826714.0,1.000,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
279536,82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...,0.0,NaN,NaN,NaN,0,19.0,2.0,53434.0,28.495,20.0,NaN,NaN,0.0,1.0,NaN,NaN,172802.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091070,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,NaN,NaN,NaN,0,14.0,2.0,51329.0,1.000,13.0,4.0,142.0,1.0,1.0,NaN,NaN,352111.0
544141,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,130.0,47.0,10.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352111.0
1091094,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,NaN,NaN,NaN,1,1.0,2.0,719.0,1.000,16.0,NaN,NaN,0.0,0.0,NaN,NaN,352111.0
1091091,67a8baafafecb5408944a7d089cda0168967144c1bd41f...,364.0,NaN,NaN,NaN,1,12.0,2.0,2987.0,1.000,12.0,NaN,NaN,0.0,0.0,NaN,NaN,352111.0


In [62]:
all_data.to_csv('./combined_data.csv')